In [31]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [33]:
import numpy as np
import pandas as pd
from math import log, sqrt, exp, erf
import numba
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
from scipy.stats import norm

In [34]:
df = pd.read_csv("winemag-data-130k-v2.csv")
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [35]:
df["price"] = df["price"].fillna(df["price"].mean())

def points_per_dollar_py(points, price):
    return points / price

def compute_py(df):
    return df.apply(lambda row: points_per_dollar_py(row["points"], row["price"]), axis=1)

%timeit compute_py(df.head(10000))

25.8 ms ± 714 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
%%cython
import numpy as np
cimport numpy as np
cpdef double points_per_dollar_cy(double points, double price):
    return points / price

cpdef compute_cython(double[:] points, double[:] prices):
    cdef Py_ssize_t n = points.shape[0]
    cdef Py_ssize_t i
    cdef double[:] result = np.empty(n, dtype="float64")
    for i in range(n):
        result[i] = points_per_dollar_cy(points[i], prices[i])
    return result

In [37]:
points_arr = df["points"].to_numpy(dtype="float64")
price_arr = df["price"].to_numpy(dtype="float64")

%timeit compute_cython(points_arr[:10000], price_arr[:10000])


10.6 μs ± 112 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [38]:
@numba.jit(nopython=True)
def compute_numba(points, prices):
    n = len(points)
    result = np.empty(n, dtype=np.float64)
    for i in range(n):
        result[i] = points[i] / prices[i]
    return result

%timeit compute_numba(points_arr[:10000], price_arr[:10000])


8.63 μs ± 2.26 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


2

In [39]:
pip install yfinance

/Users/lidaharyton/.cache/uv/builds-v0/.tmpIOi37w/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
apple = yf.Ticker("AAPL")
data = apple.history(period="1y")

S0 = data["Close"].iloc[-1]
print("Apple stock price:", S0)

Apple stock price: 255.4600067138672


In [52]:
S0 = 255.46    # ціна акції Apple
K = 260.0      # страйк
T = 30/365     # 30 днів до експірації
r = 0.05       # річна безризикова ставка
sigma = 0.229   # імпліцитна волатильність ~24%

print(f"Params: K={K}, T={T}, r={r}, sigma={sigma}")

Params: K=260.0, T=0.0821917808219178, r=0.05, sigma=0.24


In [42]:
def black_scholes_py(S0, K, T, r, sigma, option="call"):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option == "call":
        return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)

In [43]:
%%cython
import numpy as np
cimport numpy as np
from libc.math cimport log, sqrt, exp
from scipy.stats import norm

cpdef double black_scholes_cy(double S0, double K, double T, double r, double sigma, option="call"):
    cdef double d1, d2
    d1 = (log(S0 / K) + (r + 0.5 * sigma * sigma) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    if option == "call":
        return S0 * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)
    else:
        return K * exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)


In [44]:
@numba.jit(nopython=True)
def norm_cdf(x):
    return 0.5 * (1.0 + erf(x / sqrt(2.0)))

@numba.jit(nopython=True)
def black_scholes_numba(S0, K, T, r, sigma, call=True):
    d1 = (log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    if call:
        return S0 * norm_cdf(d1) - K * exp(-r * T) * norm_cdf(d2)
    else:
        return K * exp(-r * T) * norm_cdf(-d2) - S0 * norm_cdf(-d1)

In [45]:
%timeit black_scholes_py(S0, K, T, r, sigma)
%timeit black_scholes_cy(S0, K, T, r, sigma)
%timeit black_scholes_numba(S0, K, T, r, sigma)

38.2 μs ± 981 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
37.3 μs ± 1.53 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
171 ns ± 5.91 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
